# Nonimal data experiment
* [ER-MLP](#ER-MLP)
* [NTN](#NTN)

In [1]:
from core.knowledge_graph import KnowledgeGraph
from core.link_predict_utils import *

import tensorflow as tf
%pylab nbagg --no-import-all

%load_ext autoreload
%autoreload 2

test_percent = 0.1
database = KnowledgeGraph()
database.read_data_from_txt('data/kin_nominal')
database.spilt_train_valid_test(0, test_percent)
num_entities = database.number_of_entities()
num_relations = database.number_of_relations()

Populating the interactive namespace from numpy and matplotlib


## ER-MLP

In [2]:
with tf.Graph().as_default(),tf.Session() as sess:
    saver = tf.train.import_meta_graph('saved_model/er_mlp_model-kin_nominal.meta')
    saver.restore(sess, 'saved_model/er_mlp_model-kin_nominal')

    batch = sess.graph.get_tensor_by_name('Placeholders/batch:0')
    predicts = sess.graph.get_tensor_by_name('Output/predicts:0')

    for i in range(23, 127):
        test0 = []
        test1 = []
        for j in range(127):
            test0.append((i,j,0))
            test1.append((i,j,1))

        test_predicts = sess.run(predicts, {batch: test0})

        height = np.argsort(-1 * test_predicts)
        print('%s has height %s, confidence %f %f' % (
               database.entity_index_to_name(i), database.entity_index_to_name(height[0]),
               test_predicts[height[0]], test_predicts[height[1]]))

        test_predicts = sess.run(predicts, {batch: test1})

        height = np.argsort(-1 * test_predicts)
        print('%s has stature %s, confidence %f %f\n' % (
               database.entity_index_to_name(i), database.entity_index_to_name(height[0]),
               test_predicts[height[0]], test_predicts[height[1]]))

INFO:tensorflow:Restoring parameters from saved_model/er_mlp_model-kin_nominal
Person0 has height 171_cm, confidence 0.788575 0.533910
Person0 has stature 171_cm, confidence 0.843202 0.583153

Person1 has height 171_cm, confidence 0.796768 0.600181
Person1 has stature 171_cm, confidence 0.833920 0.559063

Person2 has height 167_cm, confidence 0.859236 0.769105
Person2 has stature 167_cm, confidence 0.913180 0.819466

Person3 has height 162_cm, confidence 0.545726 0.462050
Person3 has stature 162_cm, confidence 0.627679 0.481420

Person4 has height 170_cm, confidence 0.866419 0.826360
Person4 has stature 170_cm, confidence 0.879716 0.761888

Person5 has height 171_cm, confidence 0.801080 0.516068
Person5 has stature 171_cm, confidence 0.849440 0.494583

Person6 has height taller_than_180_cm, confidence 0.443230 0.391416
Person6 has stature taller_than_180_cm, confidence 0.602382 0.525365

Person7 has height 171_cm, confidence 0.797476 0.721610
Person7 has stature 171_cm, confidence 0.86

## NTN

In [3]:
def fill_feed_dict(input_list_r, data_list_r, input_list, data_list, num_relations):
    feed_dict = {}
    for (var, value) in zip(input_list_r, data_list_r):
        for r in range(num_relations):
            feed_dict[var[r]] = value[r]

    for (var, value) in zip(input_list, data_list):
        feed_dict[var] = value

    return feed_dict

In [4]:
with tf.Graph().as_default(), tf.Session() as sess:
    saver = tf.train.import_meta_graph('saved_model/ntn_model-kin_nominal.meta')
    saver.restore(sess, 'saved_model/ntn_model-kin_nominal')

    num_relations = database.number_of_relations()
    batch_input = [sess.graph.get_tensor_by_name('Placeholders/batch_%d:0' % r)
                   for r in range(num_relations)]
    r_empty_input = [sess.graph.get_tensor_by_name('Placeholders/empty_%d:0' % r)
                     for r in range(num_relations)]
    predicts = sess.graph.get_tensor_by_name('Output/predicts:0')

    for i in range(23, 127):
        test0 = []
        test1 = []
        for j in range(127):
            test0.append((i,j,0))
            test1.append((i,j,1))

        test_data, _, r_empty= make_split(test0, [1]*len(test0), num_relations)
        feed_dict = fill_feed_dict([batch_input, r_empty_input],
                                   [test_data, r_empty], [], [], num_relations)
        test_predicts = sess.run(predicts, feed_dict)

        height = np.argsort(-1 * test_predicts)
        print('%s has height %s, confidence %f %f' % (
               database.entity_index_to_name(i), database.entity_index_to_name(height[0]),
               test_predicts[height[0]], test_predicts[height[1]]))

        test_data, _, r_empty= make_split(test1, [1]*len(test1), num_relations)
        feed_dict = fill_feed_dict([batch_input, r_empty_input],
                                   [test_data, r_empty], [], [], num_relations)
        test_predicts = sess.run(predicts, feed_dict)

        height = np.argsort(-1 * test_predicts)
        print('%s has stature %s, confidence %f %f\n' % (
               database.entity_index_to_name(i), database.entity_index_to_name(height[0]),
               test_predicts[height[0]], test_predicts[height[1]]))

INFO:tensorflow:Restoring parameters from saved_model/ntn_model-kin_nominal
Person0 has height 172_cm, confidence 0.547715 0.545171
Person0 has stature 171_cm, confidence 0.501529 0.433968

Person1 has height shorter_than_160_cm, confidence 0.529955 0.525783
Person1 has stature 171_cm, confidence 0.495675 0.435473

Person2 has height shorter_than_160_cm, confidence 0.523633 0.503910
Person2 has stature 167_cm, confidence 0.451384 0.449598

Person3 has height shorter_than_160_cm, confidence 0.542217 0.481659
Person3 has stature shorter_than_160_cm, confidence 0.450596 0.427235

Person4 has height shorter_than_160_cm, confidence 0.505722 0.456345
Person4 has stature 167_cm, confidence 0.458077 0.442415

Person5 has height shorter_than_160_cm, confidence 0.533345 0.530803
Person5 has stature 171_cm, confidence 0.501292 0.438699

Person6 has height shorter_than_160_cm, confidence 0.498237 0.468833
Person6 has stature 167_cm, confidence 0.426099 0.416255

Person7 has height shorter_than_160

Person73 has stature shorter_than_160_cm, confidence 0.429303 0.424073

Person74 has height shorter_than_160_cm, confidence 0.532759 0.490955
Person74 has stature shorter_than_160_cm, confidence 0.423158 0.421456

Person75 has height 172_cm, confidence 0.561137 0.496951
Person75 has stature 171_cm, confidence 0.438258 0.426829

Person76 has height shorter_than_160_cm, confidence 0.483757 0.468046
Person76 has stature 167_cm, confidence 0.421445 0.417834

Person77 has height shorter_than_160_cm, confidence 0.504083 0.445110
Person77 has stature 167_cm, confidence 0.441410 0.432350

Person78 has height 175_cm, confidence 0.516587 0.497613
Person78 has stature shorter_than_160_cm, confidence 0.425792 0.404932

Person79 has height shorter_than_160_cm, confidence 0.484649 0.470615
Person79 has stature 167_cm, confidence 0.421353 0.418228

Person80 has height shorter_than_160_cm, confidence 0.504002 0.479209
Person80 has stature 167_cm, confidence 0.437599 0.432373

Person81 has height 175_c